# 1.Imports & Reading the Data

In [ ]:


# Import Packages




import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
from sklearn.linear_model import LinearRegression
import warnings
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import MinMaxScaler
from math import ceil 
import os
from sklearn.impute import KNNImputer
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.cm as cm
from minisom import MiniSom
import sompy
from sompy.visualization.mapview import View2D
from sompy.visualization.bmuhits import BmuHitsView
from sompy.visualization.hitmap import HitMapView
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN, estimate_bandwidth
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.base import clone
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
import pickle

import warnings
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)

warnings.filterwarnings(action='ignore', category=UserWarning, module='sklearn')
warnings.filterwarnings("ignore", module="sklearn.cluster._kmeans")

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", module="graphviz")





: 

In [3]:
# Read/Import dataset
import pickle
df = pickle.load(open("preprocessed_data_numerical.pkl", "rb"))

In [ ]:
metric_features = list(df.columns)
metric_features

# 2.Segmentation 

## 2.1 Defining our segmentations

In [7]:
demographics_preferences = ['customer_age', 'Recency', 'log_order_rate_per_week', 'log_amount_spent_per_week']


purchase_behavior = ['average_product_price','chain_percentage','log_vendor_count']


In [8]:
def cluster_profiles(df, label_columns, figsize, compar_titles=None):
    """
    Pass df with labels columns of one or multiple clustering labels. 
    Then specify this label columns to perform the cluster profile according to them.
    """
    if compar_titles == None:
        compar_titles = [""]*len(label_columns)
        
    sns.set()
    fig, axes = plt.subplots(nrows=len(label_columns), ncols=2, figsize=figsize, squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compar_titles):
        # Filtering df
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df.drop(drop_cols, axis=1)
        
        # Getting the cluster centroids and counts
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]
        
        # Setting Data
        pd.plotting.parallel_coordinates(centroids, label, color=sns.color_palette(), ax=ax[0])
        sns.barplot(x=label, y="counts", data=counts, ax=ax[1])

        #Setting Layout
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(len(handles))]
        ax[0].annotate(text=titl, xy=(0.95,1.1), xycoords='axes fraction') 
        ax[0].legend(handles, cluster_labels) # Adaptable to number of clusters
        ax[0].axhline(color="black", linestyle="--")
        ax[0].set_title("Cluster Means - {} Clusters".format(len(handles)))
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=-20)
        ax[1].set_xticklabels(cluster_labels)
        ax[1].set_xlabel("")
        ax[1].set_ylabel("Absolute Frequency")
        ax[1].set_title("Cluster Sizes - {} Clusters".format(len(handles)))
    
    plt.subplots_adjust(hspace=0.4, top=0.90)
    plt.suptitle("Cluster Simple Profilling")
    plt.show()

## 2.2 Segmentation 1 - Demographic preferences 

### Hierarchical + K-means

In [11]:
df_socio1 = df[demographics_preferences]

In [12]:
def get_ss(df):
    """Computes the sum of squares for all variables given a dataset
    """
    ss = np.sum(df.var() * (df.count() - 1))
    return ss  # return sum of sum of squares of each df variable

def r2(df, labels):
    sst = get_ss(df)
    ssw = np.sum(df.groupby(labels).apply(get_ss))
    return 1 - ssw/sst
    
def get_r2_scores(df, clusterer, min_k=2, max_k=10):
    """
    Loop over different values of k. To be used with sklearn clusterers.
    """
    r2_clust = {}
    for n in range(min_k, max_k):
        clust = clone(clusterer).set_params(n_clusters=n)
        labels = clust.fit_predict(df)
        r2_clust[n] = r2(df, labels)
    return r2_clust


# Set up the clusterers
kmeans = KMeans(
    init='k-means++',
    n_init=20,
    random_state=42
)

hierarchical = AgglomerativeClustering(
    metric='euclidean'
)

In [ ]:
# Obtaining the R² scores for each cluster solution on demographic preferences
r2_scores = {}
r2_scores['kmeans'] = get_r2_scores(df_socio1, kmeans)

for linkage in ['complete', 'average', 'single', 'ward']:
    r2_scores[linkage] = get_r2_scores(
        df_socio1, hierarchical.set_params(linkage=linkage)
    )

pd.DataFrame(r2_scores)

In [ ]:
# Visualizing the R² scores for each cluster solution on demographic variables
pd.DataFrame(r2_scores).plot.line(figsize=(10,7))

plt.title("Demographic Variables:\nR² plot for various clustering methods\n")
plt.legend(title="Cluster methods")
plt.xlabel("Number of clusters")
plt.ylabel("R² metric")
plt.show()

Based on the graph above is not clear if we should choose 3 or 4 clusters. The R² isn't very different from 3 to 4 clusters.

In [ ]:
kmclust = KMeans(n_clusters=3, init='k-means++', n_init=15, random_state=1)
kmclust.fit(df_socio1)

We are going to plot the silhouette_score to decide the final number of clusters.

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py

range_clusters = range(1, 11)

# Storing average silhouette metric
avg_silhouette = []
for nclus in range_clusters:
    # Skip nclus == 1
    if nclus == 1:
        continue
    
    # Create a figure
    fig = plt.figure(figsize=(13, 7))

    # Initialize the KMeans object with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    kmclust = KMeans(n_clusters=nclus, init='k-means++', n_init=15, random_state=1)
    cluster_labels = kmclust.fit_predict(df_socio1)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed clusters
    silhouette_avg = silhouette_score(df_socio1, cluster_labels)
    avg_silhouette.append(silhouette_avg)
    print(f"For n_clusters = {nclus}, the average silhouette_score is : {silhouette_avg}")

# The average silhouette plot
# The inertia plot
plt.figure(figsize=(9,5))
plt.plot(range_clusters[1:], ## Plot X-axis; Why range_clusters[1:] ? Remember we skipped k=1 in the cell above
         avg_silhouette)     ## Plot Y-axis

plt.ylabel("Average silhouette")
plt.xlabel("Number of clusters")
plt.title("Average silhouette plot over clusters in demographic variables", size=15)
plt.show()

It is not very common to see the silhouette_score decreasing as we add more clusters. However, the dataset might have a strong natural separation into 2 groups. Adding more clusters forces the algorithm to split well-formed groups, leading to lower silhouette scores.

Based on that we are going to choose 3 clusters. The difference in silhoete score between 3 and 4 clusters isn't very big. 

In [ ]:
df_socio1

In [ ]:
# Re-running the Hierarchical clustering based on the correct number of clusters
kmclust = KMeans(n_clusters=3, init='k-means++', n_init=15, random_state=1)
kmclust_labels = kmclust.fit_predict(df_socio1)
df_socio1['kmlust_labels'] = kmclust_labels

df_socio1

In [ ]:
df_socio1['kmlust_labels'].value_counts()

In [ ]:
cluster_profiles(df_socio1, ['kmlust_labels'], (20,7))

In [ ]:
df_socio1.groupby('kmlust_labels')[demographics_preferences].mean()

### SOM + K-means

In [31]:
df_somk_1 = df[demographics_preferences]

#### SOM Training

In [ ]:
np.random.seed(80)

sm = sompy.SOMFactory().build(
    df_somk_1.values, 
    mapsize=[10, 10], 
    initialization='random', 
    neighborhood='gaussian',
    training='batch',
    lattice='hexa',
    component_names=demographics_preferences
)
sm.train(n_job=1, verbose='info', train_rough_len=120, train_finetune_len=120)

The final quantization error is 0.798841, indicating the average distance between the data points and their corresponding best-matching unit (BMU) after training. Lower values generally indicate better mapping of input data onto the SOM.

#### Choosing the best number of clusters

In [ ]:
inertia = []
for n_clus in range(1,11): 
    kmclust = KMeans(n_clusters=n_clus, init='k-means++', n_init=20, random_state=80)
    kmclust.fit(df_somk_1)
    inertia.append(kmclust.inertia_)

plt.figure(figsize=(9,5))
plt.plot(range(1,11),inertia,color = 'blue')
plt.ylabel("Inertia: SSw")
plt.xlabel("Number of clusters")
plt.title("Inertia plot over clusters in", size=15)
plt.show()

Beyond 4 clusters, the decrease in inertia slows down, which suggests diminishing returns for adding more clusters.

The "elbow point" appears to be at 4 clusters, where the rate of improvement in inertia reduction becomes less pronounced.

#### U-matrix

#### HitMapView

In [ ]:
kmeans = KMeans(n_clusters=4,init='k-means++', n_init=20, random_state=42)
nodeclus_labels = kmeans.fit_predict(sm.codebook.matrix)
sm.cluster_labels = nodeclus_labels  # setting the cluster labels of sompy

hits = HitMapView(8,8,"Clustering", text_size=10)
hits.show(sm, anotate=True, onlyzeros=False, labelsize=7, cmap=cm.Greens)

plt.show()

Each hexagon represents a neuron in the SOM grid, and the assigned cluster labels (0, 1, 2, 3) are shown inside the respective cells.

#### Final clustering

In [ ]:
nodes = sm.codebook.matrix

df_somk = pd.DataFrame(nodes, columns=demographics_preferences)
df_somk['som_kmeans_labels'] = nodeclus_labels
df_somk

In [ ]:
df_somk['som_kmeans_labels'].value_counts()

In [ ]:
bmus_map = sm.find_bmu(df_somk_1.values)[0]  #get bmus for each observation in df

df_bmus = pd.DataFrame(
    np.concatenate((df_somk_1, np.expand_dims(bmus_map,1)), axis=1),
    index=df_somk_1.index, columns=np.append(df_somk_1.columns,"BMU")
)
df_bmus

In [ ]:
df_somk_1 = df_bmus.merge(df_somk['som_kmeans_labels'], 'left', left_on="BMU", right_index=True)

df_somk_1.drop('BMU', axis=1, inplace=True)

df_somk_1.groupby('som_kmeans_labels')[demographics_preferences].mean()

In [ ]:
cluster_profiles(df_somk_1, ['som_kmeans_labels'], (20,7))

### SOM + Hierarquical

In [49]:
df_socio3 = df[demographics_preferences]

#### SOM Training

In [ ]:
np.random.seed(80)

sm = sompy.SOMFactory().build(
    df_socio3.values, 
    mapsize=[10, 10], 
    initialization='random', 
    neighborhood='gaussian',
    training='batch',
    lattice='hexa',
    component_names=demographics_preferences
)
sm.train(n_job=1, verbose='info', train_rough_len=120, train_finetune_len=120)

#### Choosing the best number of clusters with Hierarchical

In [ ]:
linkage = 'ward'
distance = 'euclidean'
hclust = AgglomerativeClustering(linkage=linkage, metric=distance, distance_threshold=0, n_clusters=None)
hclust.fit_predict(df_socio3)

In [ ]:
counts = np.zeros(hclust.children_.shape[0])
n_samples = len(hclust.labels_)

# hclust.children_ contains the observation ids that are being merged together
# At the i-th iteration, children[i][0] and children[i][1] are merged to form node n_samples + i
for i, merge in enumerate(hclust.children_):
    # track the number of observations in the current cluster being formed
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            # If this is True, then we are merging an observation
            current_count += 1  # leaf node
        else:
            # Otherwise, we are merging a previously formed cluster
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count

# the hclust.children_ is used to indicate the two points/clusters being merged (dendrogram's u-joins)
# the hclust.distances_ indicates the distance between the two points/clusters (height of the u-joins)
# the counts indicate the number of points being merged (dendrogram's x-axis)
linkage_matrix = np.column_stack(
    [hclust.children_, hclust.distances_, counts]
).astype(float)

# Plot the corresponding dendrogram
sns.set()
fig = plt.figure(figsize=(11,5))
# The Dendrogram parameters need to be tuned
y_threshold = 90
dendrogram(linkage_matrix, truncate_mode='level', p=5, color_threshold=y_threshold, above_threshold_color='k')
plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
plt.title(f'Hierarchical Clustering - {linkage.title()}\'s Dendrogram')
plt.xlabel('Number of points in node (or index of point if no parenthesis)')
plt.ylabel(f'{distance.title()} Distance')
plt.show()

The threshold (red line) intersects just below a noticeable "gap" in the dendrogram.
Above this threshold, the vertical distances between clusters are much larger, meaning clusters are more distinct.


Based on this analysis, 6 clusters is the right choice.

#### HitMapView

In [ ]:
hierclust = AgglomerativeClustering(n_clusters=6, linkage='ward')
node_hier_label= hierclust.fit_predict(sm.codebook.matrix)
sm.cluster_labels = node_hier_label  # setting the cluster labels of sompy

hits  = HitMapView(6,6,"Clustering",text_size=10)
hits.show(sm, anotate=True, onlyzeros=False, labelsize=7, cmap=cm.Greens)

plt.show()

#### Final clustering

In [ ]:
nodes = sm.codebook.matrix

hnodes = pd.DataFrame(nodes, columns=demographics_preferences)
hnodes['som_hierar_labels'] = node_hier_label
hnodes

In [61]:
bmus_ = sm.find_bmu(df_socio3.values)[0]

In [ ]:
df_bmus_hc = pd.DataFrame(
    np.concatenate((df_socio3, np.expand_dims(bmus_,1)), axis=1),
    index=df_socio3.index, columns=np.append(df_socio3.columns,"BMU")
)
df_bmus_hc

In [63]:
# Get cluster labels for each observation
df_somh = df_bmus.merge(hnodes['som_hierar_labels'], 'left', left_on="BMU", right_index=True)

df_somh.drop('BMU', axis=1, inplace=True)

In [ ]:
cluster_profiles(df_somh, ['som_hierar_labels'], (20,7))

### DBSCAN

In [66]:
df_socio4 = df[demographics_preferences]

#### Choosing the right number of eps

In [ ]:
# Radius of cluster
neigh = NearestNeighbors(n_neighbors=27)
neigh.fit(df_socio4)
distances, _ = neigh.kneighbors(df_socio4)
distances = np.sort(distances[:, -1])
plt.plot(distances[10000:])
plt.show()

This plot above is typically used for determining an appropriate value for the epsilon (eps) parameter in DBSCAN clustering. The idea is to find the "elbow point" on the graph, which indicates the distance value where the curve transitions from a steep increase to a flatter slope. This point is a good candidate for the eps parameter, as it represents a natural clustering distance threshold in the data. 



We are going to zoom in as the right eps is not clear in the graph above. 

In [ ]:
plt.plot(distances[10000:])
plt.axis([18000, 22000, 0,1.6])
plt.show()

After zooming in, we can see in the graph above that the elbow point is around 1.0, so that is the number that we are going to choose for eps.

#### Choosing the right number of min_samples

A function is going to be used to evaluate the values of 
the min_samples, with the selected eps value fixed, giving a view of the variation of the number of clusters 
and their respective noise.

In [ ]:
for min_samples in range(2, 15):  
        dbscan = DBSCAN(eps=1.0, min_samples=min_samples)
        dbscan_labels = dbscan.fit_predict(df_socio4)
            
        dbscan_n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
            
        n_noise = list(dbscan_labels).count(-1)
            
        if dbscan_n_clusters > 0:
            print(f"min_samples: {min_samples}, clusters: {dbscan_n_clusters}, noise:{n_noise}")

min_samples = 3, 4 and 5 might seem a reasonable choice. We are going to see the Silhouette Score for each of them

In [ ]:
dbscan = DBSCAN(eps=1.0, min_samples=3, n_jobs=4)
dbscan_labels = dbscan.fit_predict(df_socio4)
score = silhouette_score(df_socio4.iloc[:, :-1], dbscan_labels)
print(f'Silhouette Score: {score}')
dbscan_clusters = len(np.unique(dbscan_labels))
print("Number of estimated clusters : %d" % dbscan_clusters)



In [ ]:
dbscan = DBSCAN(eps=1.0, min_samples=4, n_jobs=4)
dbscan_labels = dbscan.fit_predict(df_socio4)
score = silhouette_score(df_socio4.iloc[:, :-1], dbscan_labels)
print(f'Silhouette Score: {score}')
dbscan_clusters = len(np.unique(dbscan_labels))
print("Number of estimated clusters : %d" % dbscan_clusters)

In [ ]:
dbscan = DBSCAN(eps=1.0, min_samples=5, n_jobs=4)
dbscan_labels = dbscan.fit_predict(df_socio4)
score = silhouette_score(df_socio4.iloc[:, :-1], dbscan_labels)
print(f'Silhouette Score: {score}')
dbscan_clusters = len(np.unique(dbscan_labels))
print("Number of estimated clusters : %d" % dbscan_clusters)

#### Final clustering

In [81]:
df_dbscan = pd.concat([df_socio4, pd.Series(dbscan_labels, name='dbscan_labels', index=df_socio4.index)], axis=1)

In [ ]:
cluster_profiles(
    df = df_dbscan, 
    label_columns = ['dbscan_labels'], 
    figsize = (20, 7), 
)

### Combined_df

In [ ]:
combined_df = pd.concat([df_socio1, df_somk_1, df_somh, df_dbscan], ignore_index=True)

label_columns = ['kmlust_labels', 'som_kmeans_labels', 'som_hierar_labels', 'dbscan_labels']

cluster_profiles(combined_df, label_columns, figsize=(20, 20),compar_titles = ["Hierarchical + K-means", "SOM + K-means", "SOM + Hierarchical", "DBSCAN"])

## 2.3 Segmentation 2 - Purchase_behavior 

### Hierachical + K-means

In [87]:
df_eng1 = df[purchase_behavior]

In [88]:
def get_ss(df):
    """Computes the sum of squares for all variables given a dataset
    """
    ss = np.sum(df.var() * (df.count() - 1))
    return ss  # return sum of sum of squares of each df variable

def r2(df, labels):
    sst = get_ss(df)
    ssw = np.sum(df.groupby(labels).apply(get_ss))
    return 1 - ssw/sst
    
def get_r2_scores(df, clusterer, min_k=2, max_k=10):
    """
    Loop over different values of k. To be used with sklearn clusterers.
    """
    r2_clust = {}
    for n in range(min_k, max_k):
        clust = clone(clusterer).set_params(n_clusters=n)
        labels = clust.fit_predict(df)
        r2_clust[n] = r2(df, labels)
    return r2_clust


# Set up the clusterers
kmeans = KMeans(
    init='k-means++',
    n_init=20,
    random_state=42
)

hierarchical = AgglomerativeClustering(
    metric='euclidean'
)

In [ ]:
# Obtaining the R² scores for each cluster solution on demographic variables
r2_scores = {}
r2_scores['kmeans'] = get_r2_scores(df_eng1, kmeans)

for linkage in ['complete', 'average', 'single', 'ward']:
    r2_scores[linkage] = get_r2_scores(
        df_eng1, hierarchical.set_params(linkage=linkage)
    )

pd.DataFrame(r2_scores)

In [ ]:
# Visualizing the R² scores for each cluster solution on demographic variables
pd.DataFrame(r2_scores).plot.line(figsize=(10,7))

plt.title("Demographic Variables:\nR² plot for various clustering methods\n")
plt.legend(title="Cluster methods")
plt.xlabel("Number of clusters")
plt.ylabel("R² metric")
plt.show()

Based on the R² plot the correct number of clusters might be 3 or 4, but the choice isn't clear.

In [ ]:
kmclust = KMeans(n_clusters=4, init='k-means++', n_init=15, random_state=1)
kmclust.fit(df_eng1)

We are going to do the silhouette_score to decide the final number of clusters. 

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py

range_clusters = range(1, 11)

# Storing average silhouette metric
avg_silhouette = []
for nclus in range_clusters:
    # Skip nclus == 1
    if nclus == 1:
        continue
    
    # Create a figure
    fig = plt.figure(figsize=(13, 7))

    # Initialize the KMeans object with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    kmclust = KMeans(n_clusters=nclus, init='k-means++', n_init=15, random_state=1)
    cluster_labels = kmclust.fit_predict(df_eng1)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed clusters
    silhouette_avg = silhouette_score(df_eng1, cluster_labels)
    avg_silhouette.append(silhouette_avg)
    print(f"For n_clusters = {nclus}, the average silhouette_score is : {silhouette_avg}")

# The average silhouette plot
# The inertia plot
plt.figure(figsize=(9,5))
plt.plot(range_clusters[1:], ## Plot X-axis; Why range_clusters[1:] ? Remember we skipped k=1 in the cell above
         avg_silhouette)     ## Plot Y-axis

plt.ylabel("Average silhouette")
plt.xlabel("Number of clusters")
plt.title("Average silhouette plot over clusters", size=15)
plt.show()

Based on the graph above the choice is more clear. We are going to choose 3 clusters. The gain in silhouette score between 3 and 4 is very low. With 3 clusters, the results are simpler to interpret and visualize.

In [ ]:
# Re-running the Hierarchical clustering based on the correct number of clusters
kmclust = KMeans(n_clusters=3, init='k-means++', n_init=15, random_state=1)
kmclust_labels = kmclust.fit_predict(df_eng1)
df_eng1['kmlust_labels'] = kmclust_labels

df_eng1

In [ ]:
cluster_profiles(df_eng1, ['kmlust_labels'], (20,7))

### SOM + K-means

In [99]:
df_eng2 = df[purchase_behavior]

#### SOM Training

In [ ]:
np.random.seed(80)

sm = sompy.SOMFactory().build(
    df_eng2.values, 
    mapsize=[10, 10], 
    initialization='random', 
    neighborhood='gaussian',
    training='batch',
    lattice='hexa',
    component_names=purchase_behavior
)
sm.train(n_job=1, verbose='info', train_rough_len=120, train_finetune_len=120)

#### Choosing the best number of clusters

In [ ]:
inertia = []
for n_clus in range(1,11): 
    kmclust = KMeans(n_clusters=n_clus, init='k-means++', n_init=20, random_state=80)
    kmclust.fit(df_eng2)
    inertia.append(kmclust.inertia_)

plt.figure(figsize=(9,5))
plt.plot(range(1,11),inertia,color = 'blue')
plt.ylabel("Inertia: SSw")
plt.xlabel("Number of clusters")
plt.title("Inertia plot over clusters", size=15)
plt.show()

Beyond 3 clusters, the decrease in inertia slows down, which suggests diminishing returns for adding more clusters.

The "elbow point" appears to be at 3 clusters, where the rate of improvement in inertia reduction becomes less pronounced.

#### HitMapView

In [ ]:
kmeans = KMeans(n_clusters=3,init='k-means++', n_init=20, random_state=42)
nodeclus_labels = kmeans.fit_predict(sm.codebook.matrix)
sm.cluster_labels = nodeclus_labels  # setting the cluster labels of sompy

hits = HitMapView(8,8,"Clustering", text_size=10)
hits.show(sm, anotate=True, onlyzeros=False, labelsize=7, cmap="Pastel1")

plt.show()

#### Final clustering

In [ ]:
nodes = sm.codebook.matrix

df_somk = pd.DataFrame(nodes, columns=purchase_behavior)
df_somk['som_kmeans_labels'] = nodeclus_labels
df_somk


In [109]:
bmus_map = sm.find_bmu(df_eng2.values)[0]  #get bmus for each observation in df

In [ ]:
df_bmus = pd.DataFrame(
    np.concatenate((df_eng2, np.expand_dims(bmus_map,1)), axis=1),
    index=df_eng2.index, columns=np.append(df_eng2.columns,"BMU")
)
df_bmus

In [111]:
df_somk_1 = df_bmus.merge(df_somk['som_kmeans_labels'], 'left', left_on="BMU", right_index=True)

df_somk_1.drop('BMU', axis=1, inplace=True)

In [ ]:
df_somk_1.groupby('som_kmeans_labels')[purchase_behavior].mean()

In [ ]:
cluster_profiles(df_somk_1, ['som_kmeans_labels'], (20,7))

### SOM + Hierarquical

In [115]:
df_eng3 = df[purchase_behavior]

#### SOM Training

In [ ]:
np.random.seed(80)

sm = sompy.SOMFactory().build(
    df_eng3.values, 
    mapsize=[10, 10], 
    initialization='random', 
    neighborhood='gaussian',
    training='batch',
    lattice='hexa',
    component_names=purchase_behavior
)
sm.train(n_job=1, verbose='info', train_rough_len=120, train_finetune_len=120)

#### Choosing the best number of clusters with Hierarchical

In [ ]:
linkage = 'ward'
distance = 'euclidean'
hclust = AgglomerativeClustering(linkage=linkage, metric=distance, distance_threshold=0, n_clusters=None)
hclust.fit_predict(df_eng3)

In [ ]:
counts = np.zeros(hclust.children_.shape[0])
n_samples = len(hclust.labels_)

# hclust.children_ contains the observation ids that are being merged together
# At the i-th iteration, children[i][0] and children[i][1] are merged to form node n_samples + i
for i, merge in enumerate(hclust.children_):
    # track the number of observations in the current cluster being formed
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            # If this is True, then we are merging an observation
            current_count += 1  # leaf node
        else:
            # Otherwise, we are merging a previously formed cluster
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count

# the hclust.children_ is used to indicate the two points/clusters being merged (dendrogram's u-joins)
# the hclust.distances_ indicates the distance between the two points/clusters (height of the u-joins)
# the counts indicate the number of points being merged (dendrogram's x-axis)
linkage_matrix = np.column_stack(
    [hclust.children_, hclust.distances_, counts]
).astype(float)

# Plot the corresponding dendrogram
sns.set()
fig = plt.figure(figsize=(11,5))
# The Dendrogram parameters need to be tuned
y_threshold = 90
dendrogram(linkage_matrix, truncate_mode='level', p=5, color_threshold=y_threshold, above_threshold_color='k')
plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
plt.title(f'Hierarchical Clustering - {linkage.title()}\'s Dendrogram')
plt.xlabel('Number of points in node (or index of point if no parenthesis)')
plt.ylabel(f'{distance.title()} Distance')
plt.show()

The threshold (red line) intersects just below a noticeable "gap" in the dendrogram.
Above this threshold, the vertical distances between clusters are much larger, meaning clusters are more distinct.

Based on this analysis 6 is the right choice.

#### HitMapView

In [ ]:
hierclust = AgglomerativeClustering(n_clusters=6, linkage='ward')
node_hier_label= hierclust.fit_predict(sm.codebook.matrix)
sm.cluster_labels = node_hier_label  # setting the cluster labels of sompy

hits  = HitMapView(6,6,"Clustering",text_size=10)
hits.show(sm, anotate=True, onlyzeros=False, labelsize=7, cmap=cm.Greens)

plt.show()

#### Final Clustering

In [ ]:
nodes = sm.codebook.matrix

hnodes = pd.DataFrame(nodes, columns=purchase_behavior)
hnodes['som_hierar_labels'] = node_hier_label
hnodes

In [126]:
bmus_ = sm.find_bmu(df_eng3.values)[0]

In [ ]:
df_bmus = pd.DataFrame(
    np.concatenate((df_eng3, np.expand_dims(bmus_,1)), axis=1),
    index=df_eng3.index, columns=np.append(df_eng3.columns,"BMU")
)
df_bmus

In [128]:
# Get cluster labels for each observation
df_somh = df_bmus.merge(hnodes['som_hierar_labels'], 'left', left_on="BMU", right_index=True)

df_somh.drop('BMU', axis=1, inplace=True)

In [ ]:
cluster_profiles(df_somh, ['som_hierar_labels'], (20,7))

### DBSCAN

In [131]:
df_eng4 = df[purchase_behavior]

#### Choosing the right number of eps

In [ ]:
# Radius of cluster
neigh = NearestNeighbors(n_neighbors=27)
neigh.fit(df_eng4)
distances, _ = neigh.kneighbors(df_eng4)
distances = np.sort(distances[:, -1])
plt.plot(distances[10000:])
plt.show()

One more time the right eps isn't clear in the graph above.

In [ ]:
plt.plot(distances[10000:])
plt.axis([15000, 23000, 0,0.4])
plt.show()

After zooming in we can see that the "elbow" appears to be somewhere around the 0.25 - 0.30 range on the y-axis, which suggests that eps might be in this range. There isn't a choice that is 100% right. We are going to choose eps= 0.30 but it is a little bit subjective.

#### Choosing the right number of min_samples

In [ ]:
for min_samples in range(2, 15):  
        dbscan = DBSCAN(eps=0.30, min_samples=min_samples)
        dbscan_labels = dbscan.fit_predict(df_eng4)
            
        dbscan_n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
            
        n_noise = list(dbscan_labels).count(-1)
            
        if dbscan_n_clusters > 0:
            print(f"min_samples: {min_samples}, clusters: {dbscan_n_clusters}, noise:{n_noise}")

While increasing min_samples beyond 4 maintains the same number of clusters, it significantly increases noise points:

 - min_samples = 6 results in 33 noise points.
 - min_samples = 10 results in 66 noise points.
 - min_samples = 12 results in 101 noise points.
   
By choosing min_samples = 4, we include more borderline points in clusters while still maintaining clear groupings.

Smaller min_samples values lower the density threshold for forming clusters, resulting in additional, less meaningful clusters.

In [ ]:
dbscan = DBSCAN(eps=0.30, min_samples= 4,n_jobs=4)
dbscan_labels = dbscan.fit_predict(df_eng4)

dbscan_clusters = len(np.unique(dbscan_labels))
print("Number of estimated clusters : %d" % dbscan_clusters)

#### Final clustering

In [142]:
df_dbscan = pd.concat([df_eng4, pd.Series(dbscan_labels, name='dbscan_labels', index=df_eng4.index)], axis=1)

In [ ]:
cluster_profiles(
    df = df_dbscan, 
    label_columns = ['dbscan_labels'], 
    figsize = (20, 7), 
    
)

In [ ]:
df_dbscan.groupby('dbscan_labels')[purchase_behavior].mean()

### Combined_df

In [ ]:
combined_df = pd.concat([df_eng1, df_somk_1, df_somh, df_dbscan], ignore_index=True)


label_columns = ['kmlust_labels', 'som_kmeans_labels', 'som_hierar_labels', 'dbscan_labels']

cluster_profiles(combined_df, label_columns, figsize=(20, 20),compar_titles = ["Hierarchical + K-means", "SOM + K-means", "SOM + Hierarchical", "DBSCAN"])


# 3. Final clusters for each segmentation


In [148]:
df_socio = df[demographics_preferences]
df_3 = df[purchase_behavior]

In [ ]:
df_socio = df_socio.reset_index(drop=True)
df_3 = df_3.reset_index(drop=True)

df_analysis = pd.concat([df_socio, df_3], axis=1)
df_analysis

According to the several methods of clustering used to each segmentation, we decide that 3 clusters and 3 clusters are the best number for demographics_preferences and purchase_behavior respectively.

In [151]:
# Applying the right clustering (algorithm and number of clusters) for each perspective
hclust_socio = KMeans(
    n_clusters=3, 
    init='k-means++', 
    n_init=15, 
    random_state=1)
     
hc_labels_final = hclust_socio.fit_predict(df_socio)

# final cluster solution
kmclust4 = KMeans(
    n_clusters=3, 
    init='k-means++', 
    n_init=15, 
    random_state=1)

km_labels_final = kmclust4.fit_predict(df_3)

df['demographics_labels'] = hc_labels_final
df['purchase_behavior_labels'] = km_labels_final
#df_analysis['Value_labels'] = hc_labels_final
#df_analysis['Recency_labels'] = km_labels_final


In [ ]:
# Count label frequencies (contigency table)

pd.crosstab(df['demographics_labels'],
            df['purchase_behavior_labels'])

# 4. Merging using Hierarchical clustering

In [ ]:
# Centroids of the concatenated cluster labels
df_centroids = df.groupby(['demographics_labels', 'purchase_behavior_labels']).mean()
df_centroids

In [155]:
# Using Hierarchical clustering to merge the concatenated cluster centroids
linkage = 'ward'
hclust = AgglomerativeClustering(
    linkage=linkage, 
    metric='euclidean', 
    distance_threshold=0, 
    n_clusters=None
)
hclust_labels = hclust.fit_predict(df_centroids)

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py

# create the counts of samples under each node (number of points being merged)
counts = np.zeros(hclust.children_.shape[0])
n_samples = len(hclust.labels_)

# hclust.children_ contains the observation ids that are being merged together
# At the i-th iteration, children[i][0] and children[i][1] are merged to form node n_samples + i
for i, merge in enumerate(hclust.children_):
    # track the number of observations in the current cluster being formed
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            # If this is True, then we are merging an observation
            current_count += 1  # leaf node
        else:
            # Otherwise, we are merging a previously formed cluster
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count

# the hclust.children_ is used to indicate the two points/clusters being merged (dendrogram's u-joins)
# the hclust.distances_ indicates the distance between the two points/clusters (height of the u-joins)
# the counts indicate the number of points being merged (dendrogram's x-axis)
linkage_matrix = np.column_stack(
    [hclust.children_, hclust.distances_, counts]
).astype(float)

# Plot the corresponding dendrogram
sns.set()
fig = plt.figure(figsize=(11,5))
# The Dendrogram parameters need to be tuned
y_threshold = 3
dendrogram(linkage_matrix, 
           truncate_mode='level', 
           labels=df_centroids.index, p=5, 
           color_threshold=y_threshold, 
           above_threshold_color='k')

plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
plt.title(f'Hierarchical Clustering - {linkage.title()}\'s Dendrogram')
plt.xlabel('Number of points in node (or index of point if no parenthesis)')
plt.ylabel(f'Euclidean Distance')
plt.show()

At this level, the dendrogram splits into 4 distinct clusters before merging into larger clusters.
Beyond this point, the vertical distances between merges (heights of the black lines) increase significantly, indicating that merging clusters further would group dissimilar data points together.
Cutting the dendrogram at this level yields well-separated clusters based on the structure of the tree.

In [ ]:
# Re-running the Hierarchical clustering based on the correct number of clusters
hclust = AgglomerativeClustering(
    linkage='ward', 
    metric='euclidean', 
    n_clusters=3
)
hclust_labels = hclust.fit_predict(df_centroids)
df_centroids['hclust_labels'] = hclust_labels

df_centroids  # centroid's cluster labels

In [ ]:
# Mapper between concatenated clusters and hierarchical clusters
cluster_mapper = df_centroids['hclust_labels'].to_dict()
cluster_mapper


In [ ]:
df_ = df.copy()
# df_2 = df_analysis.copy()

# Mapping the hierarchical clusters on the centroids to the observations
df_['merged_labels'] = df_.apply(
    lambda row: cluster_mapper[
        (row['demographics_labels'], row['purchase_behavior_labels'])
    ], axis=1
)

'''df_2['merged_labels'] = df_2.apply(
    lambda row: cluster_mapper[
        (row['Value_labels'], row['Recency_labels'])
    ], axis=1
)
df_ca_analysis['merged_labels'] = df_ca_analysis.apply(
        lambda row: cluster_mapper[
        (row['Value_labels'], row['Recency_labels'])
    ], axis=1
)
# Merged cluster centroids
df_ca_analysis.groupby('merged_labels').mean()'''

In [ ]:
df_.groupby('merged_labels').mean(numeric_only=True)[metric_features]

In [ ]:
#Merge cluster contigency table
# Getting size of each final cluster
df_counts = df_.groupby('merged_labels')\
    .size()\
    .to_frame()

df_counts

In [ ]:
# Getting the product and behavior labels
df_counts = df_counts\
    .rename({v:k for k, v in cluster_mapper.items()})\
    .reset_index()
df_counts

In [ ]:

df_counts['demographics_labels'] = df_counts['merged_labels'].apply(lambda x: x[0])
df_counts['purchase_behavior_labels'] = df_counts['merged_labels'].apply(lambda x: x[1])


df_counts.pivot(values=0, index='demographics_labels', columns='purchase_behavior_labels')

In [165]:
# Setting df to have the final product, behavior and merged clusters
df = df_.copy()

In [ ]:
df.info()

# 5.Cluster Analysis 


## Cluster profiles graphs

In [ ]:
# Profilling each cluster (product, behavior, merged)
cluster_profiles(
    df = df, 
    label_columns = ['demographics_labels', 'purchase_behavior_labels', 'merged_labels'], 
    figsize = (28, 13), 
    compar_titles = ["Demographic clustering", "Purchase behavior clustering", "Merged clusters"]
)

## Profiling with the numerical features

In [ ]:
df_.groupby('merged_labels').mean(numeric_only=True)[metric_features]

### Merged Label: 0.0

#### Key Observations:
- Customers in this group show **higher-than-average `log_order_rate_per_week` (2.02)** and **`log_amount_spent_per_week` (1.89)**. This suggests that they are **high-frequency purchasers with significant spending habits**.
- They prefer **higher average product prices (0.13)** and show a **positive `log_vendor_count` (0.31)**, indicating a willingness to purchase from a wide variety of vendors.
- Negative `chain_percentage` (-0.11) suggests **less likelihood to shop at chain stores**.

#### Actionable Insights:
- **Target this group with premium products** and exclusive, high-value promotions to align with their spending habits and preferences.
- **Focus on independent vendors** and unique offerings rather than chain stores.
- Highlight products that **encourage high-frequency purchasing and repeat buying**.

---

### Merged Label: 1.0

#### Key Observations:
- Customers in this group have a **significant positive `chain_percentage` (0.60)**, indicating a **strong preference for chain stores**.
- **Lower-than-average spending** is reflected in **negative `log_order_rate_per_week` (-0.21)** and **`log_amount_spent_per_week` (-0.34)**.
- They prefer **lower average product prices (-0.29)** and show **slightly positive recency (-0.06)**, indicating **less frequent but recent purchasing**.

#### Actionable Insights:
- **Target this group with promotions at chain stores**, focusing on **discounts and bundled offers** to encourage higher spending.
- Highlight **value-for-money products** with lower price points.
- Leverage campaigns promoting **loyalty programs** or discounts for recent purchases to increase their engagement.

---

### Merged Label: 2.0

#### Key Observations:
- Customers in this group display **low `chain_percentage` (-1.19)**, suggesting a **strong preference for independent vendors or niche products**.
- They are **moderate spenders**, as reflected in **`log_order_rate_per_week` (0.36)** and **`log_amount_spent_per_week` (-0.05)**.
- Their preference for **higher-priced products (0.53)** suggests a **mix of selective yet quality-conscious purchasing**.

#### Actionable Insights:
- **Focus on unique, high-quality products** from independent vendors to match their preferences.
- Highlight **personalized offers** to increase spending without relying on chain-store promotions.
- **Target campaigns for customers seeking premium quality** and niche offerings.

---

### Refined Marketing Insights
- **For Merged Label 0.0**: Premium, high-value campaigns for independent vendor offerings with a focus on loyalty.
- **For Merged Label 1.0**: Budget-conscious promotions targeted at chain stores with a focus on driving higher engagement and purchases.
- **For Merged Label 2.0**: Highlight high-quality, niche products and personalized campaigns to encourage spending from selective buyers.


## Profiling with unused / categorical features

### Importing categoricals

In [175]:
merged_labels_analysis= df_.copy()

In [ ]:

data_cleaned_df = pd.read_csv("data_cleaned.csv")
merged_df = pd.merge(merged_labels_analysis, data_cleaned_df, how="outer")
columns_to_drop = metric_features + ['customer_id','demographics_labels', 'purchase_behavior_labels','total_amount_spent' ]
merged_df_dropped = merged_df.drop(columns=columns_to_drop, errors='ignore')
grouped_df = merged_df_dropped.groupby('merged_labels').mean().T
grouped_df

In [ ]:
# Add this cell to export the merged labels along with the numerical data
# Assuming df is your final DataFrame and merged_labels are your final cluster labels
df['cluster_labels'] = merged_labels  # Add the cluster labels to the DataFrame
pickle.dump(df, open("preprocessed_data_numerical_merged_labels.pkl", "rb"))

Marketing Insights

For Merged Label 0.0:

 - Focus on "DELIVERY" promotions, as they resonate most with this group.
 - Emphasize Asian cuisine in marketing campaigns.
 - Target customer_region_8670, customer_region_2360, and customer_region_4660.
 - Consider campaigns balanced across City_2 and City_3.

For Merged Label 1.0:

 - Highlight "FREEBIE" promotions to attract this segment.
 - Showcase a diverse range of cuisines, particularly Japanese and Italian.
 - Target customers in City_3 and regions like customer_region_2360 and customer_region_4660.
   
For Merged Label 2.0:

 - Promote "DELIVERY" campaigns, with additional focus on Italian and Japanese cuisines.
 - Concentrate marketing efforts in City_2 and customer_region_4660.
 - Leverage digital payment options (DIGI) to encourage transactions.

# 6. Cluster visualization using t-SNE

In [179]:
# This is step can be quite time consuming
two_dim = TSNE(random_state=42).fit_transform(df)

In [ ]:
# t-SNE visualization
pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c=df['merged_labels'], colormap='tab10', figsize=(15,10))
plt.show()

# 7. Feature Importance

In [182]:
def get_ss_variables(df):
    """Get the SS for each variable
    """
    ss_vars = df.var() * (df.count() - 1)
    return ss_vars

def r2_variables(df, labels):
    """Get the R² for each variable
    """
    sst_vars = get_ss_variables(df)
    ssw_vars = np.sum(df.groupby(labels).apply(get_ss_variables))
    return 1 - ssw_vars/sst_vars

In [ ]:
# We are essentially decomposing the R² into the R² for each variable
r2_variables(df[metric_features + ['merged_labels']], 'merged_labels').drop('merged_labels')

### Using a Decision Tree
We get the normalized total reduction of the criterion (gini or entropy) brought by that feature (also known as Gini importance).

In [ ]:
df.info()

In [ ]:
# Preparing the data
X = df[metric_features]
y = df.merged_labels

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fitting the decision tree
dt = DecisionTreeClassifier(random_state=42, max_depth=3)
dt.fit(X_train, y_train)
print("It is estimated that in average, we are able to predict {0:.2f}% of the customers correctly".format(dt.score(X_test, y_test)*100))

In [ ]:
# Assessing feature importance
pd.Series(dt.feature_importances_, index=X_train.columns).sort_values(ascending=False)

In [ ]:
# Visualizing the decision tree
dot_data = export_graphviz(dt, out_file=None, 
                           feature_names=X.columns.to_list(),
                           filled=True,
                           rounded=True,
                           special_characters=True)
g = graphviz.Source(dot_data)

g